In [4]:
import openai,os
from dotenv import load_dotenv
from llama_index.llms import OpenAI

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY')

openai.api_key = openai_api_key
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [6]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, Document

reader = SimpleDirectoryReader(input_files=['./pdf_files/sample2.pdf'])
docs = reader.load_data()

document = Document(text="\n\n".join([doc.text for doc in docs]))

In [10]:
from llama_index.vector_stores import ElasticsearchStore

vector_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="books"  # If this index doesn't exist, a new one is created
)

In [11]:
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank


def build_sentence_window_index(
    document, llm, vector_store, embed_model="local:BAAI/bge-small-en-v1.5"
):
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser
    )
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context, storage_context=storage_context
    )

    return sentence_index

def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [12]:
sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    vector_store=vector_store
)

query_engine = get_sentence_window_query_engine(sentence_index=sentence_index)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [13]:
resp = query_engine.query(
    "what is the article about"
)
print(resp)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The article is about a mysterious woman named Vida Winter who wants to reveal the truth about herself to the person reading the letter. The author reflects on the power of words and how they can captivate and affect a person.


In [14]:
resp = query_engine.query(
    "Did I tell him the truth"
)
print(resp)

No, you did not tell him the truth.


In [16]:
resp = query_engine.query(
    "How long did I sit on the stairs after reading the letter?"
)
print(resp)

The duration of time that the person sat on the stairs after reading the letter is unknown.


In [17]:
resp = query_engine.query(
    "How much did I know about Vida winter?"
)
print(resp)

You knew very little about Vida Winter.
